# Amazon Personalize AWS User Personalization + 上下文推荐示例


## 简介<a class="anchor" id="intro"></a>

在大部分情况下，Amazon Personalize 中的算法（称为配方）在很大程度上寻求解决不同的任务，如下所述：

1. **用户个性化** – 根据用户以前与项目的交互推荐项目。
1. **Personalized-Ranking** – 获取一个项目集合，然后使用类似 HRNN 的方法按照可能感兴趣的顺序对它们进行排序。
1. **SIMS（类似项目）**– 根据给定的项目，推荐用户也可交互的其他项目。

无论使用案例如何，所有算法都共享基于用户-项目-交互数据的学习基础，该数据由 3 个核心属性定义：

1. **UserID** – 参与交互的用户
1. **ItemID** – 用户与之交互的项目
1. **Timestamp** – 交互发生的时间


## 选择数据集或数据源<a class="anchor" id="source"></a>
[返回页首](#top)

正如我们所提到的，用户-项目-交互数据是开始使用服务的关键。这意味着我们需要寻找生成这种数据的使用案例，几个常见示例包括：

1. 视频点播应用
1. 电子商务平台
1. 社交媒体聚合器/平台

使用一些指导原则可帮助确定适合 Personalize 的问题的范围。我们建议将下面的值作为起点，不过[官方限制](https://docs.aws.amazon.com/personalize/latest/dg/limits.html)略低一些。

* 经过身份验证的用户
* 至少 50 个独立用户
* 至少 100 个独立项目
* 每个用户至少 24 次交互 

大多数情况下，这是很容易实现的，如果您在一个类别中排名较低，您通常可以通过在另一个类别中拥有更高排名来弥补。

一般来说，您的数据不会是适合 Personalize 的完美形式，需要进行一些修改才能正确结构化。此笔记本会指导您完成所有这些任务。

首先，我们将使用一个航空公司评论数据集。这是使用从 Skytrax（www.airlinequality.com ）中找到的所有用户评论创建的数据集。数据可以在 https://github.com/quankiquanki/skytrax-reviews-dataset 中找到 

In [274]:
import pandas as pd, numpy as np
import io
import scipy.sparse as ss
import json
import time
import datetime
import os
import sagemaker.amazon.common as smac
import boto3
import uuid
from botocore.exceptions import ClientError

### 导入并浏览数据集

In [275]:
data_dir = "airlines_data"
!mkdir $data_dir
!cd $data_dir && wget https://raw.githubusercontent.com/quankiquanki/skytrax-reviews-dataset/master/data/airline.csv


--2020-06-18 16:33:44--  https://raw.githubusercontent.com/quankiquanki/skytrax-reviews-dataset/master/data/airline.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.64.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.64.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34752262 (33M) [text/plain]
Saving to: ‘airline.csv’

airline.csv         100%[===================>]  33.14M   113MB/s    in 0.3s    

2020-06-18 16:33:45 (113 MB/s) - ‘airline.csv’ saved [34752262/34752262]



In [276]:
airline_df = pd.read_csv(data_dir + '/airline.csv')
airline_df.head()

,airline_name,link,title,author,author_country,date,content,aircraft,type_traveller,cabin_flown,route,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,wifi_connectivity_rating,value_money_rating,recommended
0,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,D Ito,Germany,2015-04-10,Outbound flight FRA/PRN A319. 2 hours 10 min f...,NaN,NaN,Economy,NaN,7.0,4.0,4.0,4.0,0.0,NaN,NaN,4.0,1
1,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Ron Kuhlmann,United States,2015-01-05,Two short hops ZRH-LJU and LJU-VIE. Very fast ...,NaN,NaN,Business Class,NaN,10.0,4.0,5.0,4.0,1.0,NaN,NaN,5.0,1
2,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,E Albin,Switzerland,2014-09-14,Flew Zurich-Ljubljana on JP365 newish CRJ900. ...,NaN,NaN,Economy,NaN,9.0,5.0,5.0,4.0,0.0,NaN,NaN,5.0,1
3,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Tercon Bojan,Singapore,2014-09-06,Adria serves this 100 min flight from Ljubljan...,NaN,NaN,Business Class,NaN,8.0,4.0,4.0,3.0,1.0,NaN,NaN,4.0,1
4,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,L James,Poland,2014-06-16,WAW-SKJ Economy. No free snacks or drinks on t...,NaN,NaN,Economy,NaN,4.0,4.0,2.0,1.0,2.0,NaN,NaN,2.0,0


正如我们在此处所看到的，数据集有很多列可供我们在 Amazon Personalize 中创建所需的数据集。

我们要做的第一件事是制作数据集的 2 个副本

In [277]:
a_interactions_df = airline_df.copy()
a_users_df = airline_df.copy()

## 构建交互数据集

让我们来构建交互数据集。执行以下步骤：

- 删除我们不感兴趣的列
- 创建新列以说明事件类型
- 将列重命名为更标准的命名约定以符合您的 Amazon Personalize 导入作业需要



In [278]:
# Keeping only 5 columns
a_interactions_df = a_interactions_df[['airline_name', 'author', 'date', 'cabin_flown', 'overall_rating']]
# Creating an additional column for Event Type
a_interactions_df['EVENT_TYPE']='RATING'
# Making sure the author name is unique without spaces
a_interactions_df['author'] = a_interactions_df['author'].str.replace(" ","")
# Rename the columns to a more Amazon Personalize standar notation
a_interactions_df.rename(columns = {'airline_name':'ITEM_ID', 'author':'USER_ID',
                              'date':'TIMESTAMP', 'cabin_flown': 'CABIN_TYPE', 'overall_rating': 'EVENT_VALUE'}, inplace = True) 
a_interactions_df.head()

,ITEM_ID,USER_ID,TIMESTAMP,CABIN_TYPE,EVENT_VALUE,EVENT_TYPE
0,adria-airways,DIto,2015-04-10,Economy,7.0,RATING
1,adria-airways,RonKuhlmann,2015-01-05,Business Class,10.0,RATING
2,adria-airways,EAlbin,2014-09-14,Economy,9.0,RATING
3,adria-airways,TerconBojan,2014-09-06,Business Class,8.0,RATING
4,adria-airways,LJames,2014-06-16,Economy,4.0,RATING


Amazon Personalize 支持**上下文推荐**。借助于这项功能，您可以通过在上下文中生成推荐来提高推荐的相关性，例如设备类型、位置、一天中的时间等。即使新用户/身份不明用户的过去交互是未知的，上下文信息对于用户个性化也很有用。

在我们的案例中，我们将使用**Cabin Type(客舱类型)**作为上下文来推荐最适合用户的航空公司。让我们探讨一下，在获取推荐时，我们将能够传递哪些值作为上下文


In [279]:
a_interactions_df.CABIN_TYPE.unique()

array(['Economy', 'Business Class', nan, 'Premium Economy', 'First Class'],
      dtype=object)

正如我们所看到的，数据集中的当前**Timestamp**值是一个字符串。Amazon Personalize 需要 Unix 类型的时间戳值。让我们取一个随机的时间戳值，并转换为 Unix 类型

In [280]:
# Get a random value from the timestamp column
arb_time_stamp = a_interactions_df.iloc[50]['TIMESTAMP']
# Transform this string to date time
date_time_obj = datetime.datetime.strptime(arb_time_stamp, '%Y-%m-%d')
print('Date:', date_time_obj.date())
# Get the date of this object
d = date_time_obj.date()
# Transform the date object to Unix time
unixtime = time.mktime(d.timetuple())
print('Unix Time: ', unixtime)

Date: 2004-03-18
Unix Time:  1079568000.0


现在，我们将对时间戳列中的所有值执行相同的转换

In [281]:
# Define a function
def convert_to_unix(string_date):
    date_time_obj = datetime.datetime.strptime(string_date, '%Y-%m-%d')
    d = date_time_obj.date()
    return time.mktime(d.timetuple())

# Apply this function across the Timestamp column
a_interactions_df['TIMESTAMP'] = a_interactions_df['TIMESTAMP'].apply(convert_to_unix)
a_interactions_df.head(5)

,ITEM_ID,USER_ID,TIMESTAMP,CABIN_TYPE,EVENT_VALUE,EVENT_TYPE
0,adria-airways,DIto,1.428624e+09,Economy,7.0,RATING
1,adria-airways,RonKuhlmann,1.420416e+09,Business Class,10.0,RATING
2,adria-airways,EAlbin,1.410653e+09,Economy,9.0,RATING
3,adria-airways,TerconBojan,1.409962e+09,Business Class,8.0,RATING
4,adria-airways,LJames,1.402877e+09,Economy,4.0,RATING


让我们来看看我们的一些数据集属性

In [282]:
a_interactions_df.describe()

,TIMESTAMP,EVENT_VALUE
count,4.139600e+04,36861.000000
mean,1.373950e+09,6.039527
std,5.771909e+07,3.214680
min,0.000000e+00,1.000000
25%,1.350864e+09,3.000000
50%,1.389658e+09,7.000000
75%,1.412122e+09,9.000000
max,1.438474e+09,10.000000


是否有空值？

In [283]:
a_interactions_df.isnull().values.any()

True

让我们删除这些空值，并确保没有任何空值

In [284]:
a_interactions_df = a_interactions_df.dropna()
a_interactions_df.isnull().values.any()

False

现在，我们已经为 Amazon Personalize 准备好了数据，让我们将这些数据保存到本地文件中

In [285]:
interactions_filename = "a_interactions.csv"
a_interactions_df.to_csv((data_dir + "/"+interactions_filename), index=False, float_format='%.0f')

## 构建用户数据集

让我们来构建用户数据集。执行以下步骤：

- 删除我们不感兴趣的列
- 创建一个新列，将 Nationality 作为用户元数据
- 将列重命名为更标准的命名约定以符合您的 Amazon Personalize 导入作业需要


In [286]:
# Copy the complete airlines data set
a_users_df = airline_df.copy()
# Select only interested columns
a_users_df = a_users_df[['author', 'author_country']]
# Clean up the authors string
a_users_df['author'] = a_users_df['author'].str.replace(" ","")
# Rename the columns
a_users_df.rename(columns = { 'author':'USER_ID', 'author_country':'NATIONALITY'}, inplace = True) 
# Drop any null values
a_users_df = a_users_df.dropna()
# Save your file locally
users_filename = "a_users.csv"
a_users_df.to_csv((data_dir +"/"+users_filename), index=False)

## 配置 S3 存储桶和 IAM 角色<a class="anchor" id="bucket_role"></a>
[返回页首](#top)

到目前为止，我们已下载、操作数据并将数据保存到运行此 Jupyter 笔记本的实例所连接的 Amazon EBS 实例中。但是，Amazon Personalize 需要一个 S3 存储桶充当您的数据源以及用于访问该存储桶的 IAM 角色。让我们把一切都安排好。

使用存储在此 Amazon SageMaker 笔记本底层的实例上的元数据来确定运行区域。如果您使用的是 Amazon SageMaker 以外的 Jupyter 笔记本，只需将区域定义为下面的字符串。Amazon S3 存储桶需要与我们目前创建的 Amazon Personalize 资源位于同一区域。

In [288]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


Amazon S3 存储桶名称是全局唯一的。要创建唯一的存储桶名称，下面的代码会将字符串 `personalizepoc` 附加到您的 AWS 账号。然后，它会使用此名称在前一个单元格中发现的区域创建一个存储桶。

In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
suffix = str(np.random.uniform())[4:9]
bucket_name = "personalize-user-personalization-example" + suffix
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

### 将数据上载到 S3

现在，您的 Amazon S3 存储桶已经创建，请上载包含我们的用户-项目-交互数据的 CSV 文件。 

In [294]:
interactions_filename = data_dir + '/a_interactions.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

In [295]:
user_metadata_file = data_dir + '/a_users.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(user_metadata_file).upload_file(user_metadata_file)

## 创建数据集组和交互数据集<a class="anchor" id="group_dataset"></a>
[返回页首](#top)

Amazon Personalize 的最高隔离和抽象级别是*数据集组*。存储在其中一个数据集组中的信息不会影响任何其他数据集组或从其中一个数据集组创建的模型，这些数据集组是完全隔离的。这允许您运行许多实验，是我们如何保持您的模型的私密性并仅对您的数据进行全面训练的一部分。

在导入先前准备好的数据之前，需要有一个数据集组和一个添加到其中的数据集来处理交互。

数据集组可以包含以下类型的信息：

* 用户-项目-交互
* 事件流（实时交互）
* 用户元数据
* 项目元数据

在为交互数据创建数据集组和数据集之前，让我们验证您的环境是否可以成功地与 Amazon Personalize 通信。

In [293]:
personalize = boto3.client(service_name='personalize')
personalize_runtime = boto3.client(service_name='personalize-runtime')
personalize_events = boto3.client(service_name='personalize-events')

### 创建数据集组

下面的单元格将创建一个名为 *airlines-dataset-group* + 后缀的新数据集组

In [64]:
dataset_group_name = "airlines-dataset-group-" + suffix

create_dataset_group_response = personalize.create_dataset_group(
    name = dataset_group_name
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:144386903708:dataset-group/airlines-dataset-group-55035",
  "ResponseMetadata": {
    "RequestId": "a8bb75fb-f15b-45da-997e-08eb14d7733a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:14:12 GMT",
      "x-amzn-requestid": "a8bb75fb-f15b-45da-997e-08eb14d7733a",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


在使用数据集组之前，它必须处于活跃状态。这可能需要一两分钟。执行下面的单元格并等待单元格显示 ACTIVE 状态。它每秒检查一次数据集组的状态，工作最长可达 3 小时。

In [65]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


现在，您有了一个数据集组，您可以为交互数据创建一个数据集。

# 创建数据集

### 交互数据集

首先，定义一个架构，告诉 Amazon Personalize 您正在上载的数据集的类型。根据数据集的类型，架构中需要几个保留关键字和强制关键字。更详细的信息可以在[文档](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html)中找到。

在这里，您将为需要 `USER_ID`、`ITEM_ID`、`TIMESTAMP`、`CABIN_TYPE`、`EVENT_TYPE`、`EVENT_VALUE` 和 `TIMESTAMP` 字段的交互数据创建架构。它们在架构中的定义顺序必须与它们在数据集中的出现顺序相同。

In [55]:
schema_name="airlines-interaction-schema-"+suffix

In [56]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name":"CABIN_TYPE",
            "type": "string",
            "categorical": True
        },
        {
          "name": "EVENT_TYPE",
          "type": "string"
        },
        {
          "name": "EVENT_VALUE",
          "type": "float"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = schema_name,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/airlines-interaction-schema-55035",
  "ResponseMetadata": {
    "RequestId": "4e045a61-d479-485c-93ff-6072076ccaa9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:10:34 GMT",
      "x-amzn-requestid": "4e045a61-d479-485c-93ff-6072076ccaa9",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [66]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn,
    name = "airlines-dataset-interactions-" + suffix
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:144386903708:dataset/airlines-dataset-group-55035/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "968e6cac-310a-4889-8243-e86ef90696ed",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:15:14 GMT",
      "x-amzn-requestid": "968e6cac-310a-4889-8243-e86ef90696ed",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 用户数据集

在这里，您将为需要 `USER_ID` 和 `NATIONALITY` 字段的用户数据创建架构。它们在架构中的定义顺序必须与它们在数据集中的出现顺序相同。


In [62]:
metadata_schema_name="airlines-users-schema-"+suffix

In [63]:
metadata_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "NATIONALITY",
            "type": "string",
            "categorical": True
        }
    ],
    "version": "1.0"
}

create_metadata_schema_response = personalize.create_schema(
    name = metadata_schema_name,
    schema = json.dumps(metadata_schema)
)

metadata_schema_arn = create_metadata_schema_response['schemaArn']
print(json.dumps(create_metadata_schema_response, indent=2))


{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/airlines-users-schema-55035",
  "ResponseMetadata": {
    "RequestId": "17844e2f-860d-484a-bb39-ab4a10e7b9fd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:13:50 GMT",
      "x-amzn-requestid": "17844e2f-860d-484a-bb39-ab4a10e7b9fd",
      "content-length": "93",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
dataset_type = "USERS"
create_metadata_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = metadata_schema_arn,
    name = "airlines-metadata-dataset-users-" + suffix
)

metadata_dataset_arn = create_metadata_dataset_response['datasetArn']
print(json.dumps(create_metadata_dataset_response, indent=2))

## 配置 S3 存储桶和 IAM 角色<a class="anchor" id="bucket_role"></a>

到目前为止，我们已下载、操作数据并将数据保存到运行此 Jupyter 笔记本的实例所连接的 Amazon EBS 实例中。但是，Amazon Personalize 需要一个 S3 存储桶充当您的数据源以及用于访问该存储桶的 IAM 角色。让我们把一切都安排好。

使用存储在此 Amazon SageMaker 笔记本底层的实例上的元数据来确定运行区域。如果您使用的是 Amazon SageMaker 以外的 Jupyter 笔记本，只需将区域定义为下面的字符串。Amazon S3 存储桶需要与我们目前创建的 Amazon Personalize 资源位于同一区域。

### 设置 S3 存储桶策略
Amazon Personalize 需要能够读取 S3 存储桶的内容。因此，添加可实现此操作的存储桶策略。

In [69]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy));

### 创建 IAM 角色

Amazon Personalize 需要能够承担 AWS 中的角色，以便拥有执行某些任务的权限。让我们来创建一个 IAM 角色并将所需的策略附加到该角色。下面的代码附加了非常宽松的策略；请对任何生产应用程序使用更严格的策略。

In [70]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise
        
# sometimes need to wait a bit for the role to be created
time.sleep(45)
print(role_arn)

arn:aws:iam::144386903708:role/PersonalizeS3Role-55035


# 创建数据集导入作业

## 导入交互数据<a class="anchor" id="import"></a>

之前，您创建了数据集组和数据集来存放您的信息，因此现在您将执行一个导入作业，将数据从 S3 存储桶加载到 Amazon Personalize 数据集。 

In [109]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "airlines-dataset-import-job-"+suffix,
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/airlines-dataset-import-job-14078",
  "ResponseMetadata": {
    "RequestId": "d118cf11-b568-4767-99d5-30a15871981a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:46:38 GMT",
      "x-amzn-requestid": "d118cf11-b568-4767-99d5-30a15871981a",
      "content-length": "121",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## 导入用户数据<a class="anchor" id="import"></a>

之前，您创建了数据集组和数据集来存放您的信息，因此现在您将执行一个导入作业，将数据从 S3 存储桶加载到 Amazon Personalize 数据集。 

In [110]:
create_metadata_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "airlines-users-metadata-dataset-import-job-"+suffix,
    datasetArn = metadata_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, user_metadata_file)
    },
    roleArn = role_arn
)

metadata_dataset_import_job_arn = create_metadata_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_metadata_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/airlines-users-metadata-dataset-import-job-14078",
  "ResponseMetadata": {
    "RequestId": "679d9401-568d-45e0-ba8c-df8f1574228d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:46:40 GMT",
      "x-amzn-requestid": "679d9401-568d-45e0-ba8c-df8f1574228d",
      "content-length": "136",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 等待数据集导入作业进入 ACTIVE 状态

在使用数据集之前，导入作业必须处于活跃状态。执行下面的单元格并等待单元格显示 ACTIVE 状态。它每秒检查一次导入作业的状态，工作最长可达 3 小时。

导入数据可能需要一些时间，具体取决于数据集的大小。在此演示中，数据导入作业已预先执行。

In [111]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [112]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = metadata_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


当数据集导入处于活跃状态时，您就可以开始使用 AWS User Personalization 配方构建模型了。

## 创建解决方案<a class="anchor" id="solutions"></a>
[返回页首](#top)

在此笔记本中，我们将使用以下配方创建解决方案：

1. aws-user-personalization


在 Amazon Personalize 中，算法的特定变体称为配方。不同的配方适用于不同的情况。经过训练的模型称为解决方案，每个解决方案可以有许多版本，这些版本与模型训练时给定的数据量相关。

首先，我们将列出支持的所有配方。这将允许您选择一个配方并使用它来构建模型。

In [113]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims
arn:aws:personalize:::recipe/aws-user-personalization


输出只是简介中提到的所有算法的 JSON 表示。

接下来，我们将选择特定的配方并使用它们构建模型。

### AWS User Personalization

AWS User Personalization 是您可以使用的更高级的推荐模型之一，它可让您根据用户行为实时更新推荐。它还往往优于其他方法，如协同筛选。这个配方的训练时间最长，所以让我们先从这个配方开始。

对于我们使用航空公司评论数据的使用案例，我们可以使用 AWS User Personalization，根据用户之前的艺术家标记行为向用户推荐航空公司。请记住，我们使用标记数据来表示用户和艺术家之间的积极交互。

首先，通过在上面的配方列表中找到 ARN 来选择配方。

In [114]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

#### 创建解决方案

首先，使用配方创建一个解决方案。尽管您在此步骤中提供了数据集 ARN，但模型尚未经过训练。将模型视为标识符，而不是经过训练的模型。

请注意，我们在此处激活了 HPO。这在下列情况下是一个好主意 


In [ ]:
create_solution_response = personalize.create_solution(
    name = "airlines-user-personalization-solution-HPO-"+suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
    performHPO=True
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### 创建解决方案版本

拥有解决方案之后，您需要创建一个版本来完成模型训练。训练过程可能需要一段时间才能完成（超过 25 分钟），并且使用我们的数据集进行训练时，采用此配方时平均需要 40 分钟。通常，我们会使用循环进行轮询，直到任务完成。但是，该任务会阻止其他单元格执行，这里的目标是创建许多模型并快速部署它们。因此，我们将为笔记本中的所有解决方案设置 While 循环。在那里，您还可以找到在 AWS 控制台中查看进度的说明。

In [117]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/airlines-hrnn-metadata-solution-HPO-14078/54a6c563",
  "ResponseMetadata": {
    "RequestId": "148a0fbd-5465-4619-ac90-449c0ef23b73",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 22:01:32 GMT",
      "x-amzn-requestid": "148a0fbd-5465-4619-ac90-449c0ef23b73",
      "content-length": "127",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [118]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

## 评估解决方案版本<a class="anchor" id="eval"></a>
[返回页首](#top)

通过该笔记本训练所有的解决方案应该不会超过一个小时。在训练过程中，我们建议花时间详细阅读各种算法（配方）及其行为。这也是一个绝佳时机，可以考虑如何将数据输入系统以及您希望看到的结果的替代方案。

解决方案创建完成后，下一步是获取评估指标。Personalize 根据训练数据的子集计算这些指标。下图说明了 Personalize 拆分数据的方式。给定 10 个用户，每个用户有 10 次交互（一个圆圈代表一次交互），交互根据时间戳从最早到最晚进行排序。Personalize 使用来自 90% 用户的所有交互数据（蓝色圆圈）来训练解决方案版本，剩下的 10% 用于评估。对于剩余 10% 中的每个用户，用户 90% 的交互数据（绿色圆圈）被用作调用训练模型的输入。其余 10% 的数据（橙色圆圈）与模型产生的输出进行比较，并用于计算评估指标。

![个性化指标](static/imgs/personalize_metrics.png)

我们建议阅读[文档](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)了解指标，但为了方便起见，我们还复制了以下文档的部分内容。

您需要了解 Personalize 中有关评估的以下术语：

* *相关推荐*是指与特定用户的测试数据中的值相匹配的推荐。
* *排名*是指推荐项目在推荐列表中的位置。位置 1（列表顶部）假定与用户最相关。
* *查询*是指 GetRecommendations 调用的内部等效项。

Personalize 生成的指标包括：
- **coverage**：训练数据（包括项目和交互数据集）中来自所有查询的唯一推荐项目数占唯一项目总数的比例。
- **mean_reciprocal_rank_at_25**：对所有查询提出的前 25 项推荐中，第一项相关推荐的[倒数排名的平均值](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)。如果您对单个排名最高的推荐感兴趣，则此指标是合适的。
- **normalized_discounted_cumulative_gain_at_K**：折损增益假设推荐列表中较下面的推荐的相关性低于较上面的推荐。因此，每项推荐都被一个因数折损（为推荐赋予较低的权重），具体取决于推荐的位置。为了产生 K 处的[累积折损增益](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)（DCG），将前 K 个推荐中的每个相关折损推荐加在一起。归一化折损累积增益（NDCG）是 DCG 除以理想 DCG，使得 NDCG 在 0-1 之间。（理想 DCG 是按相关性对前 K 个推荐进行排序。） Amazon Personalize 使用 1/log(1 + position) 权重因子，其中列表顶部为位置 1。此指标奖励出现在列表顶部附近的相关项目，因为列表的顶部通常会更受关注。
- **precision_at_K**：前 K 个推荐中相关推荐的数量除以 K。此指标奖励相关项目的精确推荐。

让我们来看看此笔记本中生成的每个解决方案的评估指标。*请注意，由于 LastFM 数据集的质量不同，您的结果可能与此笔记本文本中描述的结果不同。* 

### AWS User Personalizatioin 指标

首先，检索 AWS User Personalization解决方案版本的评估指标。

In [120]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))


{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/airlines-hrnn-metadata-solution-HPO-14078/54a6c563",
  "metrics": {
    "coverage": 0.4046,
    "mean_reciprocal_rank_at_25": 0.2035,
    "normalized_discounted_cumulative_gain_at_10": 0.2909,
    "normalized_discounted_cumulative_gain_at_25": 0.3174,
    "normalized_discounted_cumulative_gain_at_5": 0.2418,
    "precision_at_10": 0.0444,
    "precision_at_25": 0.022,
    "precision_at_5": 0.0605
  },
  "ResponseMetadata": {
    "RequestId": "156a6e70-152b-4940-8b17-048252419fd0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 23:02:24 GMT",
      "x-amzn-requestid": "156a6e70-152b-4940-8b17-048252419fd0",
      "content-length": "424",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


# 从解决方案创建活动

## 创建活动<a class="anchor" id="create"></a>

活动是托管解决方案版本；可以查询推荐的端点。通过估计吞吐量（用户每秒的个性化请求）来设置定价。部署活动时，您可以设置每秒最小吞吐量（TPS）值。与 AWS 中的许多服务一样，此服务将根据需求自动扩展，但如果延迟至关重要，您可能希望提前预置以满足更大的需求。对于此 POC 和演示，所有最小吞吐量阈值均设置为 1。有关更多信息，请参阅[定价页面](https://aws.amazon.com/personalize/pricing/)。

让我们开始部署活动。

### AWS User Personalization

为您的 AWS User Personalization 解决方案版本部署活动。部署一个活动大约需要 10 分钟。通常，我们会使用循环进行轮询，直到任务完成。但是，该任务会阻止其他单元格执行，并且这里的目标是创建多个活动。因此，我们将在笔记本中进一步为所有活动设置 While 循环。在那里，您还可以找到在 AWS 控制台中查看进度的说明。

In [122]:
create_campaign_response = personalize.create_campaign(
    name = "airlines-metadata-campaign-"+suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 2,    
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:144386903708:campaign/airlines-metadata-campaign-14078",
  "ResponseMetadata": {
    "RequestId": "4c882630-86aa-4c5d-accd-75225f9804a4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 23:25:37 GMT",
      "x-amzn-requestid": "4c882630-86aa-4c5d-accd-75225f9804a4",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [123]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


### AWS User Personalization

AWS User Personalization 是 Amazon Personalize 提供的更高级算法之一。该算法支持根据特定用户过去的行为对项目进行个性化，并且可以接收实时事件，以便在无需再训练的情况下更改对用户的推荐。

由于 AWS User Personalization 算法依赖于用户抽样，因此我们加载所需的数据，并选择 3 个随机用户。

In [262]:
users_df = pd.read_csv(data_dir + '/a_users.csv')
# Render some sample data
users_df.sample(5)

,USER_ID,NATIONALITY
3918,JHartley,United Kingdom
27477,DDriscoll,United Kingdom
19563,BrianElliott,United Kingdom
22989,AHornbuckle,Australia
35724,CMoon,United Kingdom


现在，我们为上面的 3 个随机用户提供推荐。在此之后，我们将探索实时交互，然后再进行个性化排名。

同样，我们还要创建辅助函数，在一个很好的数据框中呈现结果。

#### API 调用结果

In [165]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    
#   Context Recommendations
    context_options = ['None','Economy', 'Business Class','Premium Economy', 'First Class']
    
    for context in context_options:
        # Get the recommendations
        if context=='none':
            get_recommendations_response = personalize_runtime.get_recommendations(
                campaignArn = campaign_arn,
                userId = str(user_id),
            )
        else:
            get_recommendations_response = personalize_runtime.get_recommendations(
                campaignArn = campaign_arn,
                userId = str(user_id),
                context = {
                  'CABIN_TYPE': context
                }
            )
        # Build a new dataframe of recommendations
        item_list = get_recommendations_response['itemList']
        recommendation_list = []
        for item in item_list:
            recommendation_list.append(item['itemId'])
    #     print(recommendation_list)
        new_rec_DF = pd.DataFrame(recommendation_list, columns = [context])
        # Add this dataframe to the old one
        recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [263]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

      USER_ID     NATIONALITY
37013    RDow  United Kingdom


,None,Economy,Business Class,Premium Economy,First Class
0,british-airways,thomson-airways,british-airways,thomson-airways,united-airlines
1,united-airlines,thomas-cook-airlines,virgin-atlantic-airways,virgin-atlantic-airways,british-airways
2,thomson-airways,united-airlines,turkish-airlines,air-new-zealand,american-airlines
3,virgin-atlantic-airways,easyjet,united-airlines,british-airways,china-southern-airlines
4,air-new-zealand,monarch-airlines,china-southern-airlines,united-airlines,delta-air-lines
5,turkish-airlines,virgin-atlantic-airways,qatar-airways,thomas-cook-airlines,lufthansa
6,china-southern-airlines,british-airways,emirates,turkish-airlines,alaska-airlines
7,thomas-cook-airlines,jet2-com,air-france,monarch-airlines,virgin-atlantic-airways
8,lufthansa,lufthansa,american-airlines,china-southern-airlines,us-airways
9,american-airlines,american-airlines,lufthansa,eva-air,thomson-airways


In [264]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

      USER_ID NATIONALITY
26198   CJeff   Singapore


,None,Economy,Business Class,Premium Economy,First Class
0,philippine-airlines,philippine-airlines,cathay-pacific-airways,cathay-pacific-airways,singapore-airlines
1,cathay-pacific-airways,singapore-airlines,philippine-airlines,air-france,thai-airways
2,singapore-airlines,tigerair,malaysia-airlines,eva-air,philippine-airlines
3,ana-all-nippon-airways,jetstar-asia,srilankan-airlines,air-new-zealand,delta-air-lines
4,thai-airways,cathay-pacific-airways,thai-airways,philippine-airlines,emirates
5,air-india,airasia,singapore-airlines,klm-royal-dutch-airlines,ana-all-nippon-airways
6,china-eastern-airlines,cebu-pacific,air-india,airasia-x,china-southern-airlines
7,dragonair,scoot,emirates,qantas-airways,alaska-airlines
8,srilankan-airlines,ana-all-nippon-airways,ana-all-nippon-airways,china-southern-airlines,american-airlines
9,air-france,dragonair,asiana-airlines,united-airlines,china-eastern-airlines


在这里，我们清楚地看到，针对每个用户的推荐是不同的。如果需要缓存这些结果，您可以首先通过所有用户运行 API 调用并存储结果，也可以使用批量导出，这将在此笔记本的后面介绍。

下一个主题是实时事件。Personalize 能够侦听来自应用程序的事件，以便更新显示给用户的推荐。这在媒体工作负载（如视频点播）中尤其有用，在这种情况下，客户的意图可能会根据他们是与孩子一起观看还是独自观看而有所不同。

此外，通过此系统记录的事件将被存储，直到您删除调用，并且在您训练下个模型时，事件将与您提供的其他交互数据一起用作历史数据。

#### 实时事件

首先创建一个附加到活动的事件跟踪器。

In [150]:
response = personalize.create_event_tracker(
    name='AirlinesEventsTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:144386903708:event-tracker/d2e7ccdc
820029aa-b00c-4eff-9e6f-60830bb68508


我们将创建一些代码来模拟用户与特定项目的交互。运行此代码后，您将获得与上述结果不同的推荐。

我们首先创建一些模拟实时事件的方法。

In [200]:
session_dict = {}

def send_user_rating(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
        "itemId": str(ITEM_ID),
        "eventValue": 10,
        "cabinType": "Economy"
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
        trackingId = TRACKING_ID,
        userId= str(USER_ID),
        sessionId = session_ID,
        eventList = [{
            'sentAt': int(time.time()),
            'eventType': 'RATING',
            'properties': event_json
            }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id):
    # Interact with the airline
    # Sending a rating of 10 in Economy class for the airline with that user
    send_user_rating(USER_ID=user_id, ITEM_ID=item_id)
    
    
    #   Context Recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
        context = {
          'CABIN_TYPE': 'Economy'
        }
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        recommendation_list.append(item['itemId'])
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [item_id+'|Economy'])
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df


此时，我们还没有生成任何实时事件；我们只设置了代码。为了比较实时事件之前和之后的推荐，让我们选择一个用户并为该用户生成原始推荐。

## 使用事件跟踪器之前的推荐

In [265]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)
user_id = users[0]
recommendations_df_users

        USER_ID NATIONALITY
6313  ACrociani       Italy


,None,Economy,Business Class,Premium Economy,First Class
0,british-airways,alitalia,iberia,british-airways,american-airlines
1,alitalia,brussels-airlines,british-airways,virgin-atlantic-airways,british-airways
2,iberia,ryanair,qatar-airways,united-airlines,delta-air-lines
3,brussels-airlines,easyjet,alitalia,brussels-airlines,united-airlines
4,qatar-airways,iberia,brussels-airlines,turkish-airlines,lufthansa
5,lufthansa,aer-lingus,emirates,alitalia,emirates
6,american-airlines,aegean-airlines,lufthansa,air-france,qatar-airways
7,united-airlines,lufthansa,turkish-airlines,lufthansa,swiss-international-air-lines
8,icelandair,qatar-airways,swiss-international-air-lines,icelandair,us-airways
9,delta-air-lines,tap-portugal,oman-air,delta-air-lines,iberia


In [266]:
user_id

'ACrociani'

现在，在应用任何实时事件之前，我们有了针对此用户的推荐列表。现在，让我们选择 3 个随机艺术家，我们将模拟我们的用户交互，然后看看这会如何改变推荐。

In [267]:
# Next generate 3 random Airlines
airlines = a_interactions_df.sample(3)['ITEM_ID'].tolist()
airlines

['thai-airways', 'austrian-airlines', 'united-airlines']

In [268]:
user_recommendations_df = pd.DataFrame()
# Note this will take about 15 seconds to complete due to the sleeps
for airline in airlines:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, airline)
    time.sleep(5)
print(user_id)
user_recommendations_df

ACrociani


,thai-airways|Economy,austrian-airlines|Economy,united-airlines|Economy
0,alitalia,ryanair,brussels-airlines
1,brussels-airlines,brussels-airlines,lufthansa
2,ryanair,easyjet,turkish-airlines
3,easyjet,tap-portugal,austrian-airlines
4,iberia,aegean-airlines,tap-portugal
5,aer-lingus,turkish-airlines,alitalia
6,aegean-airlines,lufthansa,germanwings
7,lufthansa,alitalia,aegean-airlines
8,qatar-airways,iberia,swiss-international-air-lines
9,tap-portugal,air-india,air-berlin


在上面的单元格中，索引后的第一列是来自 AWS User Personalization 模型的用户默认推荐，之后的每一列都有一个标题，标题中包含他们通过实时事件与之交互的航空公司，以及该事件发生后的推荐。

行为可能不会发生很大变化；这是由于该数据集的性质相对有限。如果您想更好地理解这一点，请尝试使用随机评级来模拟随机航空公司评级，您应该会看到更明显的影响。